In [1]:
from torch.optim.adamw import adamw
!pip install -q -U "tensorflow-text"
!pip install -q tf-models-official
!pip install transformers datasets torch scikit-learn
!pip install -U accelerate
!pip install -U transformers
!pip install -U evaluate
!pip install -U datasets huggingface_hub fsspec
!pip install -U scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch

print(torch.cuda.is_available())  # Should return True if GPU is ready

True


# Dataset load and processing

In [3]:
import datasets

dataset1_raw = datasets.load_dataset("ThatOneShortGuy/SongLyrics", split="train", num_proc=3)
dataset2_raw = datasets.load_dataset("sebastiandizon/genius-song-lyrics", split="train", num_proc=3)
dataset3_raw = datasets.load_dataset("amishshah/song_lyrics", split="train", num_proc=3)

/home/vm/Documents/lu/valTeh/proj/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def language_filter(entry: dict, *extra_args):
    return entry['language'] == 'en'


dataset2_raw = dataset2_raw.filter(language_filter, num_proc=3)

In [5]:
# make so that all datasets have equal fields
print("before cleanup:")
print(dataset1_raw)
print(dataset2_raw)
print(dataset3_raw)

dataset1 = dataset1_raw
dataset2 = dataset2_raw
dataset3 = dataset3_raw

dataset1 = dataset1.remove_columns(
    ["id", "artist", 'image_url', 'release_day', 'release_month', 'release_year', 'pageviews'])
dataset2 = dataset2.remove_columns(['artist', 'year', 'views', 'features', 'id', 'language_cld3', 'language_ft'])
dataset3 = dataset3.remove_columns(['Unnamed: 0'])

dataset1 = dataset1.remove_columns(['title'])
dataset2 = dataset2.remove_columns(['title', 'language'])
dataset3 = dataset3.remove_columns(['title'])

dataset1 = dataset1.rename_column('genre', 'tag')

# language_column = ["en"] * len(dataset1)
# dataset1 = dataset1.add_column("language", language_column)

# language_column = ["en"] * len(dataset3)
# dataset3 = dataset3.add_column("language", language_column)

print("after cleanup:")
print(dataset1)
print(dataset2)
print(dataset3)

before cleanup:
Dataset({
    features: ['id', 'title', 'artist', 'lyrics', 'genre', 'image_url', 'release_day', 'release_month', 'release_year', 'pageviews'],
    num_rows: 451736
})
Dataset({
    features: ['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id', 'language_cld3', 'language_ft', 'language'],
    num_rows: 3374198
})
Dataset({
    features: ['Unnamed: 0', 'title', 'tag', 'lyrics'],
    num_rows: 3374198
})
after cleanup:
Dataset({
    features: ['lyrics', 'tag'],
    num_rows: 451736
})
Dataset({
    features: ['tag', 'lyrics'],
    num_rows: 3374198
})
Dataset({
    features: ['tag', 'lyrics'],
    num_rows: 3374198
})


In [6]:
#split datasets
dataset1_split = dataset1.train_test_split(test_size=0.1)
dataset2_split = dataset2.train_test_split(test_size=0.1)
dataset3_split = dataset3.train_test_split(test_size=0.1)

print(dataset1_split)
print(dataset2_split)
print(dataset3_split)

DatasetDict({
    train: Dataset({
        features: ['lyrics', 'tag'],
        num_rows: 406562
    })
    test: Dataset({
        features: ['lyrics', 'tag'],
        num_rows: 45174
    })
})
DatasetDict({
    train: Dataset({
        features: ['tag', 'lyrics'],
        num_rows: 3036778
    })
    test: Dataset({
        features: ['tag', 'lyrics'],
        num_rows: 337420
    })
})
DatasetDict({
    train: Dataset({
        features: ['tag', 'lyrics'],
        num_rows: 3036778
    })
    test: Dataset({
        features: ['tag', 'lyrics'],
        num_rows: 337420
    })
})


In [7]:
#combine datasets in to 2 large ones, train and test
dataset_train = datasets.concatenate_datasets(
    [dataset1_split["train"], dataset2_split["train"], dataset3_split["train"]])
dataset_test = datasets.concatenate_datasets([dataset1_split["test"], dataset2_split["test"], dataset3_split["test"]])
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['lyrics', 'tag'],
    num_rows: 6480118
})
Dataset({
    features: ['lyrics', 'tag'],
    num_rows: 720014
})


In [8]:
from collections import Counter

print("Before normalization and tag combination:")
# Group, count
train_counted = Counter(dataset_train['tag'])
test_counted = Counter(dataset_test['tag'])

# Special variable for not showing records when their count is lower than...
MIN_COUNT = 1000

# Print (and also populate tag list)
print("In train:")
for tag, count in train_counted.items():
    if (count < MIN_COUNT):
        continue
    print(f"Tag {tag}: {count} records")

print("In test:")
for tag, count in test_counted.items():
    if (count < MIN_COUNT):
        continue
    print(f"Tag {tag}: {count} records")

Before normalization and tag combination:
In train:
Tag Jazz: 5452 records
Tag Country: 38908 records
Tag Pop: 34829 records
Tag PopRock: 5523 records
Tag Rock: 75799 records
Tag RBSoul: 19265 records
Tag Metal: 6248 records
Tag Alternative: 27065 records
Tag Hip-HopRap: 28629 records
Tag Unknown: 38149 records
Tag Indie Rock: 4008 records
Tag Soul: 4793 records
Tag WorldInternational: 5063 records
Tag Latin: 8330 records
Tag Hard Rock: 2087 records
Tag DanceElectronic: 2124 records
Tag Traditional Country: 1123 records
Tag BluesFolk: 5321 records
Tag Hip-Hop: 12023 records
Tag Meme Rap: 1099 records
Tag WorldReggae: 5535 records
Tag Dance: 1566 records
Tag Rap: 5201 records
Tag SingerSongwriter: 3253 records
Tag Disco: 1534 records
Tag Classic Rock: 5251 records
Tag Punk: 1889 records
Tag Alternative Country: 1805 records
Tag Electronic: 3304 records
Tag Christian  Gospel: 1741 records
Tag ElectronicDance: 2369 records
Tag Easy ListeningVocal: 2205 records
Tag Contemporary Country: 24

In [9]:
# Manual mapping based on the results given:
# Combine similar, exclude unique (classical, regional mexicano...)
# Or vague (inspirational, world-international)

def mapping_func(row):
    # Declaring helper variables here since excluding them from here
    # Will fail num_proc attempts.
    # Constants to avoid possible misspellings
    ROCK_TAG = "rock"
    POP_TAG = "pop"
    RAP_TAG = "rap"
    COUNTRY_TAG = "country"
    RB_TAG = "rb"
    MISC_TAG = "misc"

    # Everything that is not a key in this map will be excluded.
    # Keys are normalized to lowercase to account for multiple similar records
    # (e.g. both "Rock" and "rock").
    tag_map = {
        # Rock
        "rock": ROCK_TAG,
        "metal": ROCK_TAG,
        "classic rock": ROCK_TAG,
        "hard rock": ROCK_TAG,
        "alternative": ROCK_TAG,
        "rock  roll": ROCK_TAG,
        "poprock": ROCK_TAG,
        "folk-rock": ROCK_TAG,
        "punk": ROCK_TAG,
        "indie rock": ROCK_TAG,
        # Pop
        "pop": POP_TAG,
        "disco": POP_TAG,
        "k-pop": POP_TAG,
        "j-pop": POP_TAG,
        # Rap
        "rap": RAP_TAG,
        "hip-hoprap": RAP_TAG,
        "anime rap": RAP_TAG,
        "hip-hop": RAP_TAG,
        "meme rap": RAP_TAG,
        # Country
        "country": COUNTRY_TAG,
        "outlaw country": COUNTRY_TAG,
        "contemporary country": COUNTRY_TAG,
        # R'n'B
        "rb": RB_TAG,
        "rbsoul": RB_TAG,
        "r&b": RB_TAG,
        "worldreggae": RB_TAG,
        # Misc
        # "misc": MISC_TAG,
        # "unknown": MISC_TAG,
        # "danceelectronic": MISC_TAG,
        # "electronic": MISC_TAG
    }
    # Normalize tag
    row_tag = row["tag"].strip().lower()
    if row_tag in tag_map:
        new_tag = tag_map[row_tag]
        return {"tag": new_tag}
    return {"tag": "rm"}


def mapping_func_filter(entry: dict, *extra_args):
    return entry["tag"] != 'rm'


# Normalize tags
dataset_train = dataset_train.map(mapping_func, num_proc=3)
dataset_train = dataset_train.filter(mapping_func_filter, num_proc=3)
# Normalize tags
dataset_test = dataset_test.map(mapping_func, num_proc=3)
dataset_test = dataset_test.filter(mapping_func_filter, num_proc=3)

Filter (num_proc=3): 100%|██████████| 720014/720014 [00:00<00:00, 750893.65 examples/s]


In [10]:
print("After normalization and tag combination:")
# Group, count
train_counted = Counter(dataset_train['tag'])
test_counted = Counter(dataset_test['tag'])

print("In train:")
for tag, count in train_counted.items():
    print(f"Tag {tag}: {count} records")

print("In test:")
for tag, count in test_counted.items():
    print(f"Tag {tag}: {count} records")


After normalization and tag combination:
In train:
Tag country: 197579 records
Tag pop: 2544888 records
Tag rock: 1270167 records
Tag rb: 306701 records
Tag rap: 1784096 records
In test:
Tag rock: 141263 records
Tag rap: 197916 records
Tag pop: 283024 records
Tag country: 22045 records
Tag rb: 34076 records


In [12]:
 # TODO: Balance out by reducing the count of rows in each set
# to the smallest number found in the counter after normalization.
# In our case, it will be balanced by "country",
# where only 210'155(train)/23'127(test) rows are present.
import random


def balance_dataset(dataset, label_column="label", seed=42):
    # Count examples per class
    label_counts = Counter(dataset[label_column])
    min_count = min(label_counts.values())

    print(f"Class distribution before balancing: {label_counts}")
    print(f"Downsampling to {min_count} examples per class.")

    # Group examples by label
    label_to_examples = {label: [] for label in label_counts}
    for i, example in enumerate(dataset):
        label_to_examples[example[label_column]].append(i)

    # Sample min_count examples from each class
    balanced_indices = []
    for label, indices in label_to_examples.items():
        sampled = random.Random(seed).sample(indices, min_count)
        balanced_indices.extend(sampled)

    # Select the balanced subset
    balanced_dataset = dataset.select(balanced_indices)

    # Check new distribution
    new_counts = Counter(balanced_dataset[label_column])
    print(f"Class distribution after balancing: {new_counts}")
    return balanced_dataset


# do smart balance, remove equally distributed
dataset_train_filter = balance_dataset(dataset_train, label_column="tag")



Class distribution before balancing: Counter({'pop': 2544888, 'rap': 1784096, 'rock': 1270167, 'rb': 306701, 'country': 197579})
Downsampling to 197579 examples per class.
Class distribution after balancing: Counter({'country': 197579, 'pop': 197579, 'rock': 197579, 'rb': 197579, 'rap': 197579})


In [13]:
dataset_test = balance_dataset(dataset_test, label_column="tag")

Class distribution before balancing: Counter({'pop': 283024, 'rap': 197916, 'rock': 141263, 'rb': 34076, 'country': 22045})
Downsampling to 22045 examples per class.
Class distribution after balancing: Counter({'rock': 22045, 'rap': 22045, 'pop': 22045, 'country': 22045, 'rb': 22045})


In [14]:
print("After Balancing:")
# Group, count
train_counted_filter = Counter(dataset_train_filter['tag'])
# test_counted = Counter(dataset_test['tag'])

print("In train:")
for tag, count in train_counted_filter.items():
    print(f"Tag {tag}: {count} records")

dataset_train = dataset_train_filter
# print("In test:")
# for tag, count in test_counted.items():
#     print(f"Tag {tag}: {count} records")

After Balancing:
In train:
Tag country: 197579 records
Tag pop: 197579 records
Tag rock: 197579 records
Tag rb: 197579 records
Tag rap: 197579 records


In [15]:
from numpy.random import randint
import re

# remove any occurrence of [...]
rgx = re.compile(r'(\[)(.)*(])')


def cleanup(entry):
    entry["lyrics"] = rgx.sub('', entry["lyrics"])
    return entry


dataset_train = dataset_train.map(cleanup, num_proc=3)
dataset_test = dataset_test.map(cleanup, num_proc=3)

# what we don't want
# exclude_idx = [randint(0, len(dataset_train)) for i in range(int(len(dataset_train) / 3))]

# create new dataset exluding those idx
# dataset_train = dataset_train.select(exclude_idx)

# randomly delete some entries to reduce size
# dataset_train = dataset_train.map(lambda example: {"new_sentence": example["sentence1"]}, remove_columns=["sentence1"])


Map (num_proc=3): 100%|██████████| 110225/110225 [00:02<00:00, 39960.85 examples/s]


# Tokenization

In [16]:
label_mapping = {"rb": 0, "rock": 1, "country": 2, "pop": 3, "rap": 4}
dataset_train = dataset_train.map(lambda x: {"label": label_mapping[x["tag"]]}, num_proc=3)
dataset_test = dataset_test.map(lambda x: {"label": label_mapping[x["tag"]]}, num_proc=3)



Map (num_proc=3): 100%|██████████| 110225/110225 [00:01<00:00, 72371.26 examples/s] 


In [17]:
from transformers import AutoTokenizer

# Initialize the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["lyrics"], padding="max_length", truncation=True, max_length=128)


# Apply the tokenizer to the dataset
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True, num_proc=6)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True, num_proc=6)

print(tokenized_dataset_train[0])
# save only 20% from original size
tokenized_dataset_train.save_to_disk("tokenized_dataset_train")
tokenized_dataset_test.save_to_disk("tokenized_dataset_test")



Map (num_proc=6): 100%|██████████| 110225/110225 [00:11<00:00, 9938.00 examples/s]


{'lyrics': "In a quaint caravan\nThere's a lady they call the gypsy\nShe can look in the future\nAnd drive away all your fears\n\nEverything will come right\nIf you'll only believe the gypsy\nShe could tell at a glance\nThat my heart was so full of tears\n\nShe looked at my hand and told me\nMy lover was always true\nAnd yet in my heart I knew, dear\nSomebody else was kissing you\n\nBut I'll go there again\n'Cause I want to believe the gypsy\nThat my lover is true\nAnd will come back to me someday\n\nShe looked at my hand and told me\nMy lover was always true\nAnd yet in my heart I knew, dear\nSomebody else was kissing you\nBut I'll go there again\n'Cause I want to believe the gypsy\nThat my lover is true\nAnd will come back to me someday", 'tag': 'country', 'label': 2, 'input_ids': [101, 1130, 170, 186, 6718, 10879, 25490, 1247, 112, 188, 170, 5141, 1152, 1840, 1103, 176, 1183, 12685, 1153, 1169, 1440, 1107, 1103, 2174, 1262, 2797, 1283, 1155, 1240, 10434, 5268, 1209, 1435, 1268, 1409

Saving the dataset (1/1 shards): 100%|██████████| 110225/110225 [00:00<00:00, 659331.28 examples/s]


# Balance

In [19]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# labels = dataset["train"]["label"]
# class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
# print(class_weights)